In [1]:
# !pip install datasets transformers
!pip install transformers[torch]
!pip install datasets transformers==4.28.0
!pip install accelerate -U
!apt install git-lfs
!pip install evaluate
!pip install bert_score

import transformers
import os
import json
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from IPython.display import HTML, display
from evaluate import load

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgra

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
def generate(context, model, tokenizer):
  # handles bad/empty contexts
  if not context or not isinstance(context, str):
    context = ""

  # create prompt
  prompt = context + "Agent: "

  # tokenize input (string -> tensor)
  input_ids = tokenizer([prompt], return_tensors="pt")

  # call model generate (https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/text_generation#transformers.GenerationMixin.generate)
  output_ids = model.generate(**input_ids, max_new_tokens = 36, eos_token_id = tokenizer.encode("Customer: "))

  # decode output (tensor -> string)
  pred_response = tokenizer.batch_decode(output_ids)
  return pred_response[0][len(prompt)+1:]

In [10]:
## load test data using pandas
test_data = pd.read_csv("testing_data.csv", index_col=0)
pairs_df = test_data.sample(n=1000).reset_index(drop=True)
display(pairs_df)
pairs_df.to_csv("temp.csv")

,context,response
0,"Hello! How can I help you today? Customer: Hi,...",Or are you asking for an extension?
1,"Hello, how can I help you today? Customer: I n...",I just filled out that you wanted the address ...
2,Good afternoon! How can I be of assistance tod...,I've pulled up your account.
3,"Hi, how can I help you? Customer: I will like ...",Anything else I can help you with?
4,"AcmeBrands, how may I help you? Customer: Cred...",Can I help you with anything else?
...,...,...
995,Hello. How can i help you today? Customer: My ...,and unfortunately this was not a mistake. You ...
996,"Hello, thank you for contacting AcmeBrand. How...",Was there anything else I could help with today?
997,"Hi, How can I help you today? Customer: Hi, I'...",Perfect!
998,"Customer: Hello Hello, how can i help you Cust...",You want to adjust delivery time?


In [11]:
## look into AutoModelForCausalLM.from_pretrained()
model_path = "EfazAhmed/distilgpt2-finetuned-2"
model = AutoModelForCausalLM.from_pretrained(model_path)
model.config.pad_token_id = model.config.eos_token_id
tokenizer = AutoTokenizer.from_pretrained(model_path)

df = pd.DataFrame(columns=["context", "pred_response", "true_response"])

# loop over the test set and collect pred, true responses into a list
pred_response_list, true_response_list = [], pairs_df["response"].to_list()
for index, row in pairs_df.iterrows():
  if (index+1) % 10 == 0:
    print(f"{index+1}/1000")
  pred_response = generate(row["context"], model, tokenizer)
  pred_response_list.append(pred_response)
  df.loc[len(df.index)] = [row["context"], pred_response, row["response"]]

# Evaluation metric: https://huggingface.co/spaces/evaluate-metric/bertscore (use the f1 scores from the results)
bertscore = load("bertscore")
results = bertscore.compute(predictions=pred_response_list, references=true_response_list, rescale_with_baseline=True, lang='en')
print(results)

print(f"Precision: {sum(results['precision'])/len(results['precision'])}")
print(f"Recall: {sum(results['recall'])/len(results['recall'])}")
print(f"F1 Score: {sum(results['f1'])/len(results['f1'])}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


10/1000
20/1000
30/1000
40/1000
50/1000
60/1000
70/1000
80/1000
90/1000
100/1000
110/1000
120/1000
130/1000
140/1000
150/1000
160/1000
170/1000
180/1000
190/1000
200/1000
210/1000
220/1000
230/1000
240/1000
250/1000
260/1000
270/1000
280/1000
290/1000
300/1000
310/1000
320/1000
330/1000
340/1000
350/1000
360/1000
370/1000
380/1000
390/1000
400/1000
410/1000
420/1000
430/1000
440/1000
450/1000
460/1000
470/1000
480/1000
490/1000
500/1000
510/1000
520/1000
530/1000
540/1000
550/1000
560/1000
570/1000
580/1000
590/1000
600/1000
610/1000
620/1000
630/1000
640/1000
650/1000
660/1000
670/1000
680/1000
690/1000
700/1000
710/1000
720/1000
730/1000
740/1000
750/1000
760/1000
770/1000
780/1000
790/1000
800/1000
810/1000
820/1000
830/1000
840/1000
850/1000
860/1000
870/1000
880/1000
890/1000
900/1000
910/1000
920/1000
930/1000
940/1000
950/1000
960/1000
970/1000
980/1000
990/1000
1000/1000
{'precision': [-0.06802982836961746, -0.16245993971824646, -0.20674896240234375, -0.13903433084487915, 0.248

In [12]:
df.head(5)

,context,pred_response,true_response
0,"Hello! How can I help you today? Customer: Hi,...",Agent:,Or are you asking for an extension?
1,"Hello, how can I help you today? Customer: I n...",Agent:,I just filled out that you wanted the address ...
2,Good afternoon! How can I be of assistance tod...,is my email address is cminh526@email.com. Agent:,I've pulled up your account.
3,"Hi, how can I help you? Customer: I will like ...",Agent:,Anything else I can help you with?
4,"AcmeBrands, how may I help you? Customer: Cred...",it. Agent:,Can I help you with anything else?


In [13]:
df.shape

(1000, 3)

In [14]:
df.to_csv("model_output.csv")